In [1]:
import os
import pandas as pd
import unicodedata

#### 3. signal data도 solar_name_mapping key 이름으로 저장해두기

- col name '14-15시 발전량(kWh)': 세화리, 신인천본관주차장,와산리,익산 다송리
    * 이 case들에 대해서는 추가로 col name 수정해서 저장함

In [2]:
# Creating a mapping dictionary between file names and actual power plant names
solar_name_mapping = {
    '감우리': '감우리',
    '남제주소내': '남제주소내태양광',
    '무릉리': '무릉리',
    '부산복합자재창고': '부산자재창고태양광',
    '부산본부': ['부산소내태양광#1', '부산소내태양광#2'],
    '부산수처리장': '부산 수처리장 태양광',
    '부산신항': '부산신항태양광',
    '부산역선상주차장': '부산역 선상 주차장',
    '부산운동장': '부산 운동장부지 태양광',
    #'삼척사택': '삼척사택 태양광',
    '삼척소내': ['삼척 소내태양광 #1', '삼척 소내태양광 #2', '삼척 소내태양광 #3', '삼척소내#4'],
    '세화리': '세화리',
    '송당리': '송당리 태양광',
    '신인천 1_2단계 주차장': '신인천 1,2단계 주차장 태양광',
    '신인천 북측부지': '신인천 북측부지 태양광',
    '신인천 주차장': '신인천 주차장 태양광',
    '신인천본관주차장': '신인천 소내주차장',
    '신인천소내': '신인천소내태양광',
    '신인천전망대': '신인천 전망대 태양광',
    '신인천해수구취수구': '신인천해수취수로태양광',
    '신풍리': '신풍리',
    #'안동소내': ['안동 소내태양광 #1', '안동 소내태양광#2'],
    '영월본부': '영월복합태양광',
    '영월철도부지': '영월 철도부지 태양광',
    '와산리': '와산리',
    '용수리': '제주 용수리 태양광',
    '위미2리': '위미2리태양광',
    '이천D(백사면B)': '백사면 B',
    '이천시 백사면A': '백사면 A',
    '익산 다송리': '익산다송리',
    '인천수산정수장': '인천수산정수장태양광',
    '하동공설운동장': '하동공설운동장태양광',
    '하동변전소': '하동변전소태양광',
    '하동보건소': '하동군보건소태양광',
    '하동본부': ['하동화력태양광 #1', '하동화력태양광 #2 (3개소)', '하동화력태양광 #3', '하동화력태양광 #4', '하동화력 태양광 #5', '하동소내#6'],
    '하동정수장': '하동정수장태양광',
    '하동하수처리장': '하동하수처리장태양광',
    '행원소수력': '행원소수력태양광',
    '화촌주민참여형': '화촌 솔라팜'
}
solar_name_mapping = {
    unicodedata.normalize('NFC', str(k).strip()): v 
    for k, v in solar_name_mapping.items()
}
len(solar_name_mapping)

37

In [4]:
# Signal files
path = '../../data/solar_energy/raw_data'
files = [i for i in os.listdir(path) if '태양광' in i]
files = [
    unicodedata.normalize('NFC', name.strip())
    for name in files
]
files = [f for f in files if '중부발전' not in f]
file_names = [name.split('(주)_')[1].split('태양광')[0].strip() for name in files]
file_names = [
    unicodedata.normalize('NFC', name.strip())
    for name in file_names
]
file_mapping_dict = {}
for i in range(len(files)):
    file_mapping_dict[file_names[i]] = files[i]

# Meta data
meta_data = pd.read_csv(os.path.join(path, '한국남부발전(주)_에너지원별 신재생에너지 발전설비 현황_20250312.csv'),encoding='cp949')
meta_data['발전소명'] = meta_data['발전소명'].astype(str).str.strip()
meta_data['발전소명'] = meta_data['발전소명'].apply(lambda x: unicodedata.normalize('NFC', x))


file_names.sort()
print("file_names:",len(file_names))
print(file_names[:4],'..')

print('하나의 signal file에 meta data는 여러 개 존재하는 경우 (발전호기가 2개 이상인 경우)')
list_value_keys = [k for k, v in solar_name_mapping.items() if isinstance(v, list)]
list_value_keys

file_names: 39
['감우리', '남제주소내', '무릉리', '부산복합자재창고'] ..
하나의 signal file에 meta data는 여러 개 존재하는 경우 (발전호기가 2개 이상인 경우)


['부산본부', '삼척소내', '하동본부']

In [5]:
# 1. 1:1 매핑 가능한 데이터들에 대해서 우선 저장
keys = [i for i in solar_name_mapping.keys() if i not in list_value_keys]
print('1. 1:1 매핑 가능한 데이터들:',len(keys),'\n')

1. 1:1 매핑 가능한 데이터들: 34 



In [ ]:
# 데이터 저장할 경로 정의
save_path = '../../data/solar_energy/preprocessed_data'
os.makedirs(save_path,exist_ok=True)

In [9]:

# 1. 1:1 매핑 가능한 데이터들
for key in keys:
    choose = file_mapping_dict[key]
    file = pd.read_csv(os.path.join(path, choose), encoding='cp949')
    
    try:
        # 보통의 데이터 시간대별 column name: 1,2,3,4.. 체크
        _ = file['15']
        hour_columns = [str(i) for i in range(1, 25)]
    except:
        # col name '14-15시 발전량(kWh)' 형태 처리
        hour_columns = []
        rename_dict = {}
        for col in file.columns:
            if isinstance(col, str) and '시 발전량' in col:
                # ex) '14-15시 발전량 (kWh)' → 15
                hour = col.split('-')[-1].split('시')[0].strip()
                rename_dict[col] = hour
                hour_columns.append(hour)
        file.rename(columns=rename_dict, inplace=True)
    
    # '년월일' 컬럼을 'date'로 변경하고 형식 변환
    file.rename(columns={'년월일': 'date'}, inplace=True)
    
    # 날짜 형식이 숫자인 경우 문자열로 변환 후 하이픈 추가
    if file['date'].dtype != 'object':
        file['date'] = pd.to_datetime(file['date'].astype(str), format='%Y%m%d').dt.strftime('%Y-%m-%d')
    elif file['date'].iloc[0].isdigit() and len(file['date'].iloc[0]) == 8:
        file['date'] = pd.to_datetime(file['date'], format='%Y%m%d').dt.strftime('%Y-%m-%d')
    
    # Wide 형식에서 Long 형식으로 변환
    id_vars = [col for col in file.columns if col not in hour_columns]
    melted_df = pd.melt(file, 
                        id_vars=id_vars, 
                        value_vars=hour_columns,
                        var_name='hour', 
                        value_name='value')
    
    # 시간에 ':00' 추가하여 'time' 컬럼 생성
    melted_df['time'] = melted_df['hour'].apply(lambda x: f"{int(x):02d}:00")
    melted_df.drop('hour', axis=1, inplace=True)
    
    # 날짜와 시간으로 정렬
    melted_df = melted_df.sort_values(by=['date', 'time']).reset_index(drop=True)
    
    # parquet으로 저장 (인코딩 문제 없음)
    melted_df.to_parquet(os.path.join(save_path, f'{key}.parquet'))

# 2. 발전호기 2개 이상인 경우
def save_signal_per_unit(file, key, save_path):
    # unique 호기 번호 찾기
    unique_units = file['호기'].unique()

    for unit in unique_units:
        # 해당 호기의 데이터만 추출
        unit_df = file[file['호기'] == unit].copy()
        
        # 년월일 컬럼명 변경 및 형식 변환
        unit_df.rename(columns={'년월일': 'date'}, inplace=True)
        
        # 날짜 형식이 숫자인 경우 문자열로 변환 후 하이픈 추가
        if unit_df['date'].dtype != 'object':
            unit_df['date'] = pd.to_datetime(unit_df['date'].astype(str), format='%Y%m%d').dt.strftime('%Y-%m-%d')
        elif len(unit_df) > 0 and unit_df['date'].iloc[0].isdigit() and len(unit_df['date'].iloc[0]) == 8:
            unit_df['date'] = pd.to_datetime(unit_df['date'], format='%Y%m%d').dt.strftime('%Y-%m-%d')

        # 시간 컬럼 식별
        hour_columns = []
        for col in unit_df.columns:
            if col.isdigit() or (isinstance(col, str) and col.isdigit()):
                hour_columns.append(col)
        
        # Wide 형식에서 Long 형식으로 변환
        id_vars = [col for col in unit_df.columns if col not in hour_columns]
        melted_df = pd.melt(unit_df, 
                            id_vars=id_vars, 
                            value_vars=hour_columns,
                            var_name='hour', 
                            value_name='value')
        
        # 시간에 ':00' 추가하여 'time' 컬럼 생성
        melted_df['time'] = melted_df['hour'].apply(lambda x: f"{int(x):02d}:00")
        melted_df.drop('hour', axis=1, inplace=True)
        
        # 날짜와 시간으로 정렬
        melted_df = melted_df.sort_values(by=['date', 'time']).reset_index(drop=True)
        
        # parquet으로 저장
        file_name = f"{key}_{unit}.parquet"
        melted_df.to_parquet(os.path.join(save_path, file_name))

for key in list_value_keys:
    choose = file_mapping_dict[key]
    file = pd.read_csv(os.path.join(path, choose), encoding='cp949')
    
    try:
        # 보통의 데이터 시간대별 column name: 1,2,3,4.. 체크
        _ = file['15']
        hour_columns = [str(i) for i in range(1, 25)]
    except:
        # col name '14-15시 발전량(kWh)' 형태 처리
        hour_columns = []
        rename_dict = {}
        for col in file.columns:
            if isinstance(col, str) and '시 발전량' in col:
                # ex) '14-15시 발전량 (kWh)' → 15
                hour = col.split('-')[-1].split('시')[0].strip()
                rename_dict[col] = hour
                hour_columns.append(hour)
        file.rename(columns=rename_dict, inplace=True)
    
    save_signal_per_unit(file, key, save_path)

In [ ]:
sample = pd.read_parquet('/Users/soomin/Desktop/공공데이터/renewable-power-prediction/data/solar_energy/preprocessed_data/하동본부_1.parquet')
sample

,date,호기,총량(kw),평균(kw),최대(kw),최소(kw),최대(시간별_kw),최소(시간별_kw),value,time
0,2013-01-01,1,1681.56,70.00,443.52,0,443.52,0.12,0.0,01:00
1,2013-01-01,1,1681.56,70.00,443.52,0,443.52,0.12,0.0,02:00
2,2013-01-01,1,1681.56,70.00,443.52,0,443.52,0.12,0.0,03:00
3,2013-01-01,1,1681.56,70.00,443.52,0,443.52,0.12,0.0,04:00
4,2013-01-01,1,1681.56,70.00,443.52,0,443.52,0.12,0.0,05:00
...,...,...,...,...,...,...,...,...,...,...
106603,2025-02-28,1,3129.12,130.38,551.64,0,551.64,1.80,0.0,20:00
106604,2025-02-28,1,3129.12,130.38,551.64,0,551.64,1.80,0.0,21:00
106605,2025-02-28,1,3129.12,130.38,551.64,0,551.64,1.80,0.0,22:00
106606,2025-02-28,1,3129.12,130.38,551.64,0,551.64,1.80,0.0,23:00


#### column이름에 'kw'대신 'kWh' 들어간 것들 통일하기

In [1]:
import os
import pandas as pd

def standardize_column_names(df):
    """컬럼명을 표준화하는 함수"""
    # 컬럼명 매핑 딕셔너리
    column_mapping = {
        '총량(kWh)': '총량(kw)',
        '평균(kWh)': '평균(kw)',
        '최대(kWh)': '최대(kw)',
        '최소(kWh)': '최소(kw)',
        '최대(시간별_kWh)': '최대(시간별_kw)',
        '최소(시간별_kWh)': '최소(시간별_kw)'
    }
    
    # 컬럼명 변경
    df = df.rename(columns=column_mapping)
    
    return df

path = '../../data/solar_energy/preprocessed_data'
files = [i for i in os.listdir(path) if i.endswith('.parquet')]

for f in files:
    file_path = os.path.join(path, f)
    df = pd.read_parquet(file_path)
    
    # 컬럼명 표준화
    df = standardize_column_names(df)
    
    # 변경된 데이터를 다시 저장
    df.to_parquet(file_path, index=False)
    
    print(f'{f} 파일의 컬럼명이 표준화되었습니다.')
    print(f'컬럼명: {list(df.columns)}')
    print('---')

print('모든 파일의 컬럼명 표준화 완료!')

부산운동장.parquet 파일의 컬럼명이 표준화되었습니다.
컬럼명: ['date', '호기', '총량(kw)', '평균(kw)', '최대(kw)', '최소(kw)', '최대(시간별_kw)', '최소(시간별_kw)', 'value', 'time']
---
화촌주민참여형.parquet 파일의 컬럼명이 표준화되었습니다.
컬럼명: ['date', '호기', '총량(kw)', '평균(kw)', '최대(kw)', '최소(kw)', '최대(시간별_kw)', '최소(시간별_kw)', 'value', 'time']
---
삼척소내_2.parquet 파일의 컬럼명이 표준화되었습니다.
컬럼명: ['date', '호기', '총량(kw)', '평균(kw)', '최대(kw)', '최소(kw)', '최대(시간별_kw)', '최소(시간별_kw)', 'value', 'time']
---
영월본부.parquet 파일의 컬럼명이 표준화되었습니다.
컬럼명: ['date', '호기', '총량(kw)', '평균(kw)', '최대(kw)', '최소(kw)', '최대(시간별_kw)', '최소(시간별_kw)', 'value', 'time']
---
하동변전소.parquet 파일의 컬럼명이 표준화되었습니다.
컬럼명: ['date', '호기', '총량(kw)', '평균(kw)', '최대(kw)', '최소(kw)', '최대(시간별_kw)', '최소(시간별_kw)', 'value', 'time']
---
신인천 북측부지.parquet 파일의 컬럼명이 표준화되었습니다.
컬럼명: ['date', '호기', '총량(kw)', '평균(kw)', '최대(kw)', '최소(kw)', '최대(시간별_kw)', '최소(시간별_kw)', 'value', 'time']
---
삼척소내_3.parquet 파일의 컬럼명이 표준화되었습니다.
컬럼명: ['date', '호기', '총량(kw)', '평균(kw)', '최대(kw)', '최소(kw)', '최대(시간별_kw)', '최소(시간별_kw)', 'value', 'time']
---
